# Data Loading and Basic Cleaning

Load raw weather and outage data files with minimal processing.

**Pipeline:**
1. **01_data_loading.ipynb** (this notebook) - Load and reshape raw data
2. **02_data_integration_eda.ipynb** - Merge datasets and analysis  
3. **03_visualizations.ipynb** - Data visualizations

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
np.random.seed(42)

In [2]:
import shutil
import subprocess

# Check for NVIDIA GPU
NVIDIA_SMI = shutil.which("nvidia-smi")
GPU_INFO = []

if NVIDIA_SMI:
    try:
        query = "--query-gpu=name,memory.total,driver_version"
        fmt = "--format=csv,noheader"
        out = subprocess.check_output([NVIDIA_SMI, query, fmt], text=True).strip().splitlines()
        for line in out:
            name, mem, driver = [x.strip() for x in line.split(",")]
            GPU_INFO.append({"name": name, "memory_total": mem, "driver_version": driver})
        print("NVIDIA GPU detected:")
        for i, g in enumerate(GPU_INFO):
            print(f"  [{i}] {g['name']} | {g['memory_total']} | Driver {g['driver_version']}")
    except Exception as e:
        print(f"GPU check failed: {e}")
else:
    print("No NVIDIA GPU detected (nvidia-smi not available)")

# Check PyTorch GPU support
TORCH_AVAILABLE = False
TORCH_DEVICE = None
try:
    import torch
    TORCH_AVAILABLE = True
    if torch.cuda.is_available():
        TORCH_DEVICE = torch.device("cuda")
        torch.backends.cudnn.benchmark = True
        print(f"PyTorch: GPU enabled - {torch.cuda.get_device_name(0)}")
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        TORCH_DEVICE = torch.device("mps")
        print("PyTorch: Apple MPS enabled")
    else:
        TORCH_DEVICE = torch.device("cpu")
        print("PyTorch: CPU only")
except ImportError:
    print("PyTorch not installed")

# Check TensorFlow GPU support
TF_AVAILABLE = False
TF_GPUS = []
try:
    import tensorflow as tf
    TF_AVAILABLE = True
    TF_GPUS = tf.config.list_physical_devices("GPU")
    if TF_GPUS:
        for g in TF_GPUS:
            try:
                tf.config.experimental.set_memory_growth(g, True)
            except Exception:
                pass
        print(f"⚡ TensorFlow: {len(TF_GPUS)} GPU(s) detected")
    else:
        print("💻 TensorFlow: CPU only")
except ImportError:
    print("⚠️ TensorFlow not installed")

NVIDIA GPU detected:
  [0] Tesla T4 | 15360 MiB | Driver 550.54.15
PyTorch: GPU enabled - Tesla T4
⚡ TensorFlow: 1 GPU(s) detected


## Colab Environment Setup

This section configures the environment for Google Colab with GPU support.

In [3]:
import os
import sys
from pathlib import Path

# Detect if running in Google Colab
try:
    import google.colab
    IS_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IS_COLAB = False
    print("Not running in Colab - using local environment")

# Setup directories
if IS_COLAB:
    # Mount Google Drive for persistent storage
    from google.colab import drive
    # drive.mount('/content/drive', force_remount=True)

    # Configure project paths in Drive
    PROJECT_ROOT = Path("/content/drive/MyDrive/ABT_Global/AI-Studio-Project")
    PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

    # Create data directories
    DATA_DIR = PROJECT_ROOT / "data"
    RAW_DATA_DIR = DATA_DIR / "raw"
    PROCESSED_DATA_DIR = DATA_DIR / "processed"

    RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
    PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

    print(f"Project root: {PROJECT_ROOT}")
    print(f"Raw data: {RAW_DATA_DIR}")
    print(f"Processed data: {PROCESSED_DATA_DIR}")
else:
    # Local environment paths
    PROJECT_ROOT = Path("../..").resolve()
    DATA_DIR = PROJECT_ROOT / "data"
    RAW_DATA_DIR = DATA_DIR / "raw"
    PROCESSED_DATA_DIR = DATA_DIR / "processed"
    print(f"Using local paths: {PROJECT_ROOT}")

Running in Google Colab
Project root: /content/drive/MyDrive/AI-Studio-Project
Raw data: /content/drive/MyDrive/AI-Studio-Project/data/raw
Processed data: /content/drive/MyDrive/AI-Studio-Project/data/processed


In [4]:
from pathlib import Path
import shutil

def clear_raw_data(confirm=False):
    """
    Deletes all files and subdirectories inside RAW_DATA_DIR.
    Set confirm=True to proceed, otherwise it will only report what would be deleted.
    """
    base = RAW_DATA_DIR

    # Safety checks
    if not isinstance(base, Path):
        raise ValueError("RAW_DATA_DIR is not a pathlib.Path")
    if not base.exists():
        print(f"Raw data dir does not exist: {base}")
        return
    # Ensure it's under the expected project data directory
    if DATA_DIR not in base.parents and base != DATA_DIR:
        raise ValueError(f"Refusing to clear non-project path: {base}")

    items = list(base.iterdir())
    if not items:
        print(f"Raw data dir is already empty: {base}")
        return

    print(f"Found {len(items)} items in {base}:")
    for p in items:
        print(f" - {p}")

    if not confirm:
        print("\nDry run complete. Pass confirm=True to delete.")
        return


    removed = 0
    for p in items:
        try:
            if p.is_dir():
                shutil.rmtree(p)
            else:
                p.unlink()
            removed += 1
        except Exception as e:
            print(f"Failed to remove {p}: {e}")

    print(f"\nRemoved {removed} items from {base}")

# Example usage:
clear_raw_data(confirm=True)

Raw data dir is already empty: /content/drive/MyDrive/AI-Studio-Project/data/raw


In [5]:
def download_data_if_needed():
    """
    Download raw data if running in Colab and data doesn't exist.
    For local runs, verify data is present.
    """
    if IS_COLAB:
        weather_dir = RAW_DATA_DIR / "weather" / "daily_grids"
        outage_dir = RAW_DATA_DIR / "outages"

        weather_exists = weather_dir.exists()
        outage_exists = outage_dir.exists()

        if weather_exists and outage_exists:
            print("Data found in Google Drive (weather and outages)")
            return True
        else:
            print("Data not fully available in Google Drive.")
            if not weather_exists:
                print(f"   Missing weather data → {weather_dir}")
            if not outage_exists:
                print(f"   Missing outage data  → {outage_dir}")
            print("")
            print("IMPORTANT: You can either:")
            print("   Option 1: Upload data manually to your Google Drive")
            print(f"      - Weather data → {weather_dir}")
            print(f"      - Outage data → {outage_dir}")
            print("")
            print("   Option 2: Run the automated download in the next cell")
            print("      (This will download from NOAA and Figshare)")

            weather_dir.mkdir(parents=True, exist_ok=True)
            outage_dir.mkdir(parents=True, exist_ok=True)
            return False
    else:
        weather_exists = (RAW_DATA_DIR / "weather").exists()
        outage_exists = (RAW_DATA_DIR / "outages").exists()

        if weather_exists and outage_exists:
            print("Local data directories found (weather and outages)")
            return True
        else:
            print("Warning: Data directories not fully available locally")
            if not weather_exists:
                print(f"   Missing: {RAW_DATA_DIR / 'weather'}")
            if not outage_exists:
                print(f"   Missing: {RAW_DATA_DIR / 'outages'}")
            print(f"   Expected base: {RAW_DATA_DIR}")
            return False

data_available = download_data_if_needed()

Data not fully available in Google Drive.
   Missing weather data → /content/drive/MyDrive/AI-Studio-Project/data/raw/weather/daily_grids
   Missing outage data  → /content/drive/MyDrive/AI-Studio-Project/data/raw/outages

IMPORTANT: You can either:
   Option 1: Upload data manually to your Google Drive
      - Weather data → /content/drive/MyDrive/AI-Studio-Project/data/raw/weather/daily_grids
      - Outage data → /content/drive/MyDrive/AI-Studio-Project/data/raw/outages

   Option 2: Run the automated download in the next cell
      (This will download from NOAA and Figshare)


## Automated Data Download (Optional)

**Run this cell ONLY if you need to download data from source.**

This will download:
- Weather data from NOAA daily-grids
- Outage data from Figshare

⚠️ Note: This may take 10-30 minutes and use significant bandwidth.

In [6]:
# Automated data download - adapted from your fetch scripts
import requests
import re
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Configuration
YEARS_TO_DOWNLOAD = ["2014", "2015", "2016", "2017", "2018", "2019", "2020"]
WEATHER_VARS = ["tmax", "tmin", "prcp"]  # Temperature max/min, precipitation
LEVEL = "cty"  # County level
FIGSHARE_ARTICLE_ID = 24237376  # EAGLE-I outage data

def download_weather_data(years, variables, level, output_base):
    """Download NOAA daily grids weather data"""
    BASE_URL = "https://www.ncei.noaa.gov/pub/data/daily-grids/v1-0-0/averages/"

    def list_year_files(year):
        url = f"{BASE_URL}{year}/"
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        names = re.findall(r'href="([^"/][^"]+)"', r.text)
        return [(n, url + n) for n in names]

    def passes_filters(fname, year, variables, level):
        fn = fname.lower()
        if not fn.endswith(".csv"):
            return False
        if not any(fn.startswith(v.lower() + "-") for v in variables):
            return False
        if not fn.startswith(str(year)[2:4]):  # Check year in filename
            m = re.search(r"-(\d{6})-", fn)
            if not m or not m.group(1).startswith(str(year)):
                return False
        if f"-{level.lower()}-scaled" not in fn:
            return False
        return True

    def download_file(url, dest):
        dest.parent.mkdir(parents=True, exist_ok=True)
        if dest.exists() and dest.stat().st_size > 0:
            return f"[SKIP] {dest.name}"

        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            total = int(r.headers.get("content-length", 0))
            with open(dest, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    if chunk:
                        f.write(chunk)
        return f"[OK] {dest.name}"

    total_downloaded = 0
    for year in years:
        print(f"\n>> Fetching weather data for {year}...")
        entries = list_year_files(year)
        keep = [(name, url) for (name, url) in entries
                if passes_filters(name, year, variables, level)]

        if not keep:
            print(f"   WARNING: No files matched for {year}")
            continue

        print(f"   Found {len(keep)} files to download")
        outdir = output_base / "weather" / "daily_grids" / year

        with ThreadPoolExecutor(max_workers=4) as ex:
            futures = []
            for name, url in keep:
                dest = outdir / name
                futures.append(ex.submit(download_file, url, dest))

            for fut in tqdm(as_completed(futures), total=len(futures), desc=f"   {year}"):
                try:
                    result = fut.result()
                    total_downloaded += 1
                except Exception as e:
                    print(f"   ERROR: {e}")

    return total_downloaded

def download_outage_data(years, article_id, output_base):
    """Download EAGLE-I outage data from Figshare"""
    API_URL = f"https://api.figshare.com/v2/articles/{article_id}"

    print(f"\n>> Fetching outage data from Figshare...")
    r = requests.get(API_URL, timeout=60)
    r.raise_for_status()
    files = r.json().get("files", [])

    print(f"   Found {len(files)} files in Figshare article")
    if files:
        print(f"   File names: {[f.get('name', 'unnamed') for f in files[:5]]}")  # Show first 5

    pat = re.compile(r"eaglei_outages_(\d{4})\.csv$", re.IGNORECASE)
    years_set = set(str(y) for y in years)

    downloaded = 0
    for f in files:
        name = f.get("name", "")
        m = pat.search(name)
        if not m:
            continue
        yr = m.group(1)
        if yr not in years_set:
            continue

        url = f["download_url"]
        dest = output_base / "outages" / yr / name
        dest.parent.mkdir(parents=True, exist_ok=True)

        if dest.exists() and dest.stat().st_size > 0:
            print(f"   [SKIP] {name} (already exists)")
            continue

        print(f"   >> Downloading {name}...")
        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            total = int(r.headers.get("content-length", 0))
            with open(dest, "wb") as file:
                with tqdm(total=total, unit="B", unit_scale=True, desc=f"   {name}") as pbar:
                    for chunk in r.iter_content(chunk_size=1024*1024):
                        if chunk:
                            file.write(chunk)
                            pbar.update(len(chunk))
        print(f"   [OK] {name} downloaded")
        downloaded += 1

    return downloaded

# Main download execution
if not data_available:
    print("Starting automated data download...\n")
    print(f"Download location: {RAW_DATA_DIR}\n")

    # Download weather data
    weather_count = download_weather_data(YEARS_TO_DOWNLOAD, WEATHER_VARS, LEVEL, RAW_DATA_DIR)
    print(f"\n[SUCCESS] Weather: Downloaded {weather_count} files")

    # Download outage data
    outage_count = download_outage_data(YEARS_TO_DOWNLOAD, FIGSHARE_ARTICLE_ID, RAW_DATA_DIR)
    print(f"[SUCCESS] Outages: Downloaded {outage_count} files")

    print(f"\nDownload complete! Total: {weather_count + outage_count} files")
    print(f"Data saved to: {RAW_DATA_DIR}")

    # Verify download
    data_available = download_data_if_needed()
else:
    print("[OK] Data already available - skipping download")

Starting automated data download...

Download location: /content/drive/MyDrive/AI-Studio-Project/data/raw


>> Fetching weather data for 2014...
   Found 36 files to download


   2014: 100%|██████████| 36/36 [00:22<00:00,  1.60it/s]



>> Fetching weather data for 2015...
   Found 36 files to download


   2015: 100%|██████████| 36/36 [00:20<00:00,  1.75it/s]



>> Fetching weather data for 2016...
   Found 36 files to download


...

   [OK] eaglei_outages_2018.csv downloaded
   >> Downloading eaglei_outages_2019.csv...


   eaglei_outages_2019.csv: 100%|██████████| 1.10G/1.10G [00:52<00:00, 21.2MB/s]


   [OK] eaglei_outages_2019.csv downloaded
   >> Downloading eaglei_outages_2020.csv...


   eaglei_outages_2020.csv: 100%|██████████| 1.17G/1.17G [01:12<00:00, 16.1MB/s]

   [OK] eaglei_outages_2020.csv downloaded
[SUCCESS] Outages: Downloaded 7 files

Download complete! Total: 259 files
Data saved to: /content/drive/MyDrive/AI-Studio-Project/data/raw
Data found in Google Drive (weather and outages)


## Data Download & Setup

Download raw data files if running in Colab or verify local data availability.

## GPU & Hardware Detection

Check available GPU and configure accelerators for training.

In [7]:
# Configuration for data source - now uses dynamic paths from setup
YEARS = ["2014", "2015", "2016", "2017", "2018", "2019", "2020"]

ANALYSIS_YEAR = "2014"  # Change this to analyze different years
USE_SINGLE_YEAR = False  # Set to False to load all years

if USE_SINGLE_YEAR:
    print(f"Loading data for {ANALYSIS_YEAR} only...")
    years_to_load = [ANALYSIS_YEAR]
else:
    print(f"Loading all years: {', '.join(YEARS)}...")
    years_to_load = YEARS

weather_files = {'tmax': [], 'tmin': [], 'prcp': []}

# Use RAW_DATA_DIR from setup instead of hardcoded path
for year in years_to_load:
    weather_dir = RAW_DATA_DIR / "weather" / "daily_grids" / year
    if weather_dir.exists():
        weather_files['tmax'].extend(sorted(weather_dir.glob(f"tmax-{year}*.csv")))
        weather_files['tmin'].extend(sorted(weather_dir.glob(f"tmin-{year}*.csv")))
        weather_files['prcp'].extend(sorted(weather_dir.glob(f"prcp-{year}*.csv")))
        print(f"  Found weather data for {year}")
    else:
        print(f"  Warning: No weather data found for {year} at {weather_dir}")

print(f"\nTotal weather files: tmax={len(weather_files['tmax'])}, tmin={len(weather_files['tmin'])}, prcp={len(weather_files['prcp'])}")

# Define column names (NOAA data has no header row)
cols = ['level', 'fips_code', 'county_name', 'year', 'month', 'variable'] + [f'day_{i:02d}' for i in range(1, 32)]

def load_weather_variable(file_list, var_name):
    """Load and combine weather files for one variable"""
    if not file_list:
        print(f"Warning: No files found for {var_name}")
        return pd.DataFrame()

    dfs = []
    for file in file_list:
        try:
            df = pd.read_csv(file, header=None, names=cols)
            dfs.append(df)
        except Exception as e:
            print(f"Error loading {file}: {e}")

    if dfs:
        combined = pd.concat(dfs, ignore_index=True)
        print(f"  Loaded {var_name}: {len(combined):,} rows from {len(file_list)} files")
        return combined
    else:
        return pd.DataFrame()

# Load each variable type
print("\nLoading weather data...")
tmax_df = load_weather_variable(weather_files['tmax'], 'TMAX')
tmin_df = load_weather_variable(weather_files['tmin'], 'TMIN')
prcp_df = load_weather_variable(weather_files['prcp'], 'PRCP')

# Combine all weather variables
weather_df = pd.concat([tmax_df, tmin_df, prcp_df], ignore_index=True)
print(f"Combined weather data: {len(weather_df):,} rows")

# Load outage data for selected years
print("\nLoading outage data...")
outage_dfs = []
outage_df = pd.DataFrame()

for year in years_to_load:
    outage_file = RAW_DATA_DIR / "outages" / year / f"eaglei_outages_{year}.csv"
    if outage_file.exists():
        df = pd.read_csv(outage_file)
        outage_dfs.append(df)
        print(f"  Loaded outages for {year}: {len(df):,} rows")
    else:
        print(f"  Warning: No outage data found for {year} at {outage_file}")

if outage_dfs:
    outage_df = pd.concat(outage_dfs, ignore_index=True)
    print(f"Combined outage data: {len(outage_df):,} rows")
else:
    print("Warning: No outage data loaded! Using empty DataFrame.")

print(f"\nData loading complete!")
print(f"   Weather data shape: {weather_df.shape}")
print(f"   Outage data shape: {outage_df.shape}")
print(f"   Years included: {', '.join(years_to_load)}")

Loading all years: 2014, 2015, 2016, 2017, 2018, 2019, 2020...
  Found weather data for 2014
  Found weather data for 2015
  Found weather data for 2016
  Found weather data for 2017
  Found weather data for 2018
  Found weather data for 2019
  Found weather data for 2020

Total weather files: tmax=84, tmin=84, prcp=84

Loading weather data...
  Loaded TMAX: 260,988 rows from 84 files
  Loaded TMIN: 260,988 rows from 84 files
  Loaded PRCP: 260,988 rows from 84 files
Combined weather data: 782,964 rows

Loading outage data...
  Loaded outages for 2014: 1,689,460 rows
  Loaded outages for 2015: 12,939,157 rows
  Loaded outages for 2016: 13,306,024 rows
  Loaded outages for 2017: 15,078,364 rows
  Loaded outages for 2018: 21,776,806 rows
  Loaded outages for 2019: 24,074,122 rows
  Loaded outages for 2020: 25,545,517 rows
Combined outage data: 114,409,450 rows

Data loading complete!
   Weather data shape: (782964, 37)
   Outage data shape: (114409450, 5)
   Years included: 2014, 2015, 2

In [ ]:
# 1. Check weather data structure and geographic identifiers
print("Weather data shape:", weather_df.shape)
weather_df.columns.tolist()

In [15]:
print("Unique variables:", weather_df['variable'].unique())
print("Unique FIPS codes:", weather_df['fips_code'].nunique())
print("Weather data sample:")
weather_df.head(10)

Unique variables: ['TMAX' 'TMIN' 'PRCP']
Unique FIPS codes: 3107
Weather data sample:


,level,fips_code,county_name,year,month,variable,day_01,day_02,day_03,day_04,day_05,day_06,day_07,day_08,day_09,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,cty,1001,AL: Autauga,2014,1,TMAX,7.92,9.14,12.62,4.46,9.72,15.71,-0.88,-1.97,6.69,10.84,19.29,19.64,17.45,14.06,16.13,9.87,10.23,11.34,10.92,13.94,18.29,10.75,6.47,7.05,2.19,12.79,16.24,15.80,-1.58,-1.08,7.44
1,cty,1003,AL: Baldwin County,2014,1,TMAX,8.78,15.52,15.37,6.91,14.86,19.52,4.86,1.16,8.87,14.76,21.41,23.40,19.44,16.80,18.44,12.56,12.71,16.45,13.77,17.31,19.15,15.94,9.69,12.68,4.84,14.51,15.99,19.10,4.29,0.47,9.43
2,cty,1005,AL: Barbour County,2014,1,TMAX,9.29,10.71,12.95,5.90,7.93,15.71,5.06,-0.36,7.52,10.74,19.20,20.52,18.15,16.82,17.28,11.87,10.16,14.80,10.84,16.15,17.71,13.14,7.20,8.90,2.76,14.36,15.69,16.64,1.67,-1.43,4.62
3,cty,1007,AL: Bibb County,2014,1,TMAX,7.73,9.92,11.60,4.13,10.67,14.23,-2.16,-1.52,7.36,12.80,18.47,18.39,16.72,12.72,15.63,9.42,10.63,9.98,10.68,13.43,17.33,8.85,5.43,5.70,2.63,12.15,15.06,13.81,-2.93,-0.89,7.52
4,cty,1009,AL: Blount County,2014,1,TMAX,8.72,10.35,9.26,3.82,9.61,11.35,-6.26,-3.19,6.70,11.47,16.95,16.87,15.50,11.71,14.54,7.54,8.99,7.27,8.94,12.09,15.35,5.87,3.03,1.89,1.27,8.71,12.52,8.60,-3.11,-0.87,7.05
5,cty,1011,AL: Bullock County,2014,1,TMAX,8.51,9.34,12.17,5.45,7.29,15.39,3.07,-1.32,7.09,9.71,18.87,20.04,17.56,15.91,16.46,10.30,9.67,13.25,10.75,15.22,17.44,11.80,6.41,7.75,2.28,13.80,15.61,15.24,-0.02,-1.89,5.84
6,cty,1013,AL: Butler County,2014,1,TMAX,8.35,10.44,13.64,6.75,10.88,16.49,0.93,-0.23,8.07,12.02,20.41,21.50,18.06,14.78,16.35,10.71,11.28,13.05,12.58,15.44,18.34,12.67,8.00,8.40,3.15,14.05,16.13,15.99,-0.22,0.69,8.54
7,cty,1015,AL: Calhoun County,2014,1,TMAX,7.58,10.70,10.52,4.38,8.35,12.68,-2.70,-3.28,7.89,10.64,17.91,18.48,16.42,12.92,15.07,8.57,9.30,9.26,9.16,12.84,17.55,9.30,3.71,3.71,2.27,8.66,14.61,12.35,-2.75,-0.69,6.99
8,cty,1017,AL: Chambers County,2014,1,TMAX,8.95,8.96,10.27,5.37,7.67,13.46,2.23,-0.71,8.08,10.06,18.68,18.94,16.66,15.48,14.84,9.04,10.16,11.04,10.34,15.42,17.16,9.84,5.70,5.60,3.64,11.16,15.35,13.83,-1.08,-0.11,8.74
9,cty,1019,AL: Cherokee County,2014,1,TMAX,6.88,10.59,10.07,2.56,7.55,11.80,-3.15,-4.07,7.23,9.87,16.44,17.94,16.07,11.72,14.86,8.46,8.42,8.70,8.27,12.69,17.62,9.03,3.15,2.32,1.66,7.60,14.17,11.28,-3.84,0.03,5.82


In [16]:
# 2. Check outage data structure and geographic identifiers
print("\nOutage data shape:", outage_df.shape)
outage_df.columns.tolist()


Outage data shape: (114409450, 7)


['fips_code',
 'county',
 'state',
 'customers_out',
 'run_start_time',
 'date',
 'outage_occurred']

In [17]:
print("Outage data sample:")
outage_df.head(100)


Outage data sample:


,fips_code,county,state,customers_out,run_start_time,date,outage_occurred
0,1037,Coosa,Alabama,12.0,2014-11-01 04:00:00,2014-11-01,1
1,1051,Elmore,Alabama,7.0,2014-11-01 04:00:00,2014-11-01,1
2,1109,Pike,Alabama,1.0,2014-11-01 04:00:00,2014-11-01,1
3,1121,Talladega,Alabama,31.0,2014-11-01 04:00:00,2014-11-01,1
4,4017,Navajo,Arizona,1.0,2014-11-01 04:00:00,2014-11-01,1
...,...,...,...,...,...,...,...
95,19113,Linn,Iowa,2.0,2014-11-01 04:00:00,2014-11-01,1
96,20045,Douglas,Kansas,45.0,2014-11-01 04:00:00,2014-11-01,1
97,20091,Johnson,Kansas,2.0,2014-11-01 04:00:00,2014-11-01,1
98,20173,Sedgwick,Kansas,1.0,2014-11-01 04:00:00,2014-11-01,1


## Data Processing Summary

**Weather Data:**
- Source: 84 monthly CSV files (2014-2020)
- Reshaped from long to wide format
- Variables: TMAX, TMIN, PRCP

**Outage Data:**  
- Source: 7 annual CSV files (2014-2020)
- Deduplicated by county and date
- Created binary outage indicator

In [18]:
# Weather data cleaning pipeline with memory optimization
print("Cleaning weather data...")
print(f"Initial data shape: {weather_df.shape}")
print(f"Memory usage: {weather_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")

# Check if weather_df is empty
if weather_df.empty:
    print("ERROR: weather_df is empty. Please run cell 12 first to load the data.")
    raise ValueError("weather_df is not loaded")

# Process each weather variable separately to reduce memory footprint
unique_vars = weather_df['variable'].unique()
print(f"Processing {len(unique_vars)} weather variables: {unique_vars}\n")

weather_cleaned_parts = []

for var in unique_vars:
    print(f"[{var}] Filtering data...")
    var_df = weather_df[weather_df['variable'] == var].copy()

    print(f"[{var}] Melting from wide to long format...")
    var_long = var_df.melt(
        id_vars=['fips_code', 'county_name', 'year', 'month'],
        value_vars=[f'day_{i:02d}' for i in range(1, 32)],
        var_name='day',
        value_name=var.lower()
    )

    # Free memory
    del var_df

    print(f"[{var}] Creating date column...")
    var_long['day_num'] = var_long['day'].str.extract('(\d+)').astype(int)
    var_long['date'] = pd.to_datetime(
        var_long[['year', 'month', 'day_num']].rename(columns={'day_num': 'day'}),
        errors='coerce'
    )

    # Remove invalid dates and unnecessary columns
    var_long = var_long.dropna(subset=['date'])
    var_long = var_long[['fips_code', 'county_name', 'date', var.lower()]]

    print(f"[{var}] Processed {len(var_long):,} records")
    weather_cleaned_parts.append(var_long)

    # Free memory
    del var_long

# Merge all variables on fips_code, county_name, and date
print("\nMerging weather variables...")
weather_pivot = weather_cleaned_parts[0]

for i, part in enumerate(weather_cleaned_parts[1:], 1):
    print(f"  Merging variable {i+1}/{len(weather_cleaned_parts)}...")
    weather_pivot = weather_pivot.merge(
        part,
        on=['fips_code', 'county_name', 'date'],
        how='outer'
    )
    del part

# Clean up
del weather_cleaned_parts

# Sort by date for better organization
print("Sorting by date...")
weather_pivot = weather_pivot.sort_values(['fips_code', 'date']).reset_index(drop=True)

print(f"\nWeather data cleaned: {weather_pivot.shape}")
print(f"   Memory usage: {weather_pivot.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Date range: {weather_pivot['date'].min()} to {weather_pivot['date'].max()}")
print("\nSample of cleaned weather data:")
weather_pivot.head(20)

Cleaning weather data...
Initial data shape: (782964, 37)
Memory usage: 331.49 MB

Processing 3 weather variables: ['TMAX' 'TMIN' 'PRCP']

[TMAX] Filtering data...
[TMAX] Melting from wide to long format...
[TMAX] Creating date column...
[TMAX] Processed 7,944,599 records
[TMIN] Filtering data...
[TMIN] Melting from wide to long format...
[TMIN] Creating date column...
[TMIN] Processed 7,944,599 records
[PRCP] Filtering data...
[PRCP] Melting from wide to long format...
[PRCP] Creating date column...
[PRCP] Processed 7,944,599 records

Merging weather variables...
  Merging variable 2/3...
  Merging variable 3/3...
Sorting by date...

Weather data cleaned: (7944599, 6)
   Memory usage: 810.27 MB
   Date range: 2014-01-01 00:00:00 to 2020-12-31 00:00:00

Sample of cleaned weather data:


,fips_code,county_name,date,tmax,tmin,prcp
0,1001,AL: Autauga,2014-01-01,7.92,3.12,0.00
1,1001,AL: Autauga,2014-01-02,9.14,4.07,7.54
2,1001,AL: Autauga,2014-01-03,12.62,-4.65,1.41
3,1001,AL: Autauga,2014-01-04,4.46,-5.44,0.00
4,1001,AL: Autauga,2014-01-05,9.72,-4.71,0.00
5,1001,AL: Autauga,2014-01-06,15.71,-3.26,2.39
6,1001,AL: Autauga,2014-01-07,-0.88,-11.17,0.00
7,1001,AL: Autauga,2014-01-08,-1.97,-11.40,0.00
8,1001,AL: Autauga,2014-01-09,6.69,-8.77,0.00
9,1001,AL: Autauga,2014-01-10,10.84,-1.10,3.90


In [ ]:
# 2.2 Outage data cleaning pipeline
print("Cleaning outage data...")

# Convert run_start_time to datetime
outage_df['run_start_time'] = pd.to_datetime(outage_df['run_start_time'])

# Extract date (without time) for matching with weather data
outage_df['date'] = outage_df['run_start_time'].dt.date
outage_df['date'] = pd.to_datetime(outage_df['date'])  # Convert back to datetime for consistency

# Create binary outage indicator
outage_df['outage_occurred'] = 1  # All rows in outage data represent outages

# Remove duplicates based on fips, date (keep first occurrence per county per day)
print(f"Before deduplication: {outage_df.shape}")
outage_clean = outage_df.drop_duplicates(subset=['fips_code', 'date'], keep='first')
print(f"After deduplication: {outage_clean.shape}")

# Check for missing FIPS codes
print(f"Missing FIPS codes: {outage_clean['fips_code'].isnull().sum()}")

print("Sample of cleaned outage data:")
outage_clean.head(10)

## Export Cleaned Data

In [14]:
# Save cleaned datasets using dynamic paths
print(f"💾 Saving cleaned datasets to: {PROCESSED_DATA_DIR}")

# Ensure directory exists
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Save cleaned weather data
weather_file = PROCESSED_DATA_DIR / "weather_cleaned.csv"
weather_pivot.to_csv(weather_file, index=False)
print(f"✅ Weather data saved: {weather_file}")
print(f"   Shape: {weather_pivot.shape[0]:,} rows × {weather_pivot.shape[1]} columns")

# Save cleaned outage data
outage_file = PROCESSED_DATA_DIR / "outages_cleaned.csv"
outage_clean.to_csv(outage_file, index=False)
print(f"✅ Outage data saved: {outage_file}")
print(f"   Shape: {outage_clean.shape[0]:,} rows × {outage_clean.shape[1]} columns")

print("\n🎉 Data export complete - ready for downstream notebooks!")

💾 Saving cleaned datasets to: /content/drive/MyDrive/AI-Studio-Project/data/processed
✅ Weather data saved: /content/drive/MyDrive/AI-Studio-Project/data/processed/weather_cleaned.csv
   Shape: 7,944,599 rows × 6 columns
✅ Outage data saved: /content/drive/MyDrive/AI-Studio-Project/data/processed/outages_cleaned.csv
   Shape: 3,993,154 rows × 7 columns

🎉 Data export complete - ready for downstream notebooks!


## Data Dictionary

**weather_cleaned.csv**
- `fips_code`: County FIPS code
- `county_name`: County name  
- `date`: Date
- `tmax`: Maximum temperature (°C)
- `tmin`: Minimum temperature (°C)
- `prcp`: Precipitation (mm)

**outages_cleaned.csv**
- `fips_code`: County FIPS code
- `date`: Date
- `outage_occurred`: Binary indicator (1=outage, 0=no outage)
- `customers_out`: Number of customers affected
- `run_start_time`: Original outage timestamp